In [1]:
from pyspark.sql import functions as F

# Stop current Spark session
try:
    spark.stop()
except:
    pass

from pyspark.sql import SparkSession

# Give Spark way more memory since you have 32GB RAM available
spark = SparkSession.builder \
    .appName("TimeSeriesForecast") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

print("Spark restarted with 12GB driver and executor memory")

# Read data
df = spark.read.parquet('../notebooks/data/train.parquet')
df.show(2)
print(f"Total rows: {df.count()}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/24 01:56:16 WARN Utils: Your hostname, daniels-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.88.117 instead (on interface en0)
25/12/24 01:56:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/24 01:56:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark restarted with 12GB driver and executor memory
+----+------+------+--------------------+----------+---------+-----+-----------+-------+----------+----------+--------------------+----------------+--------------+---------------+--------------+-----------+------------+-----+----+-------------+
|type| state|  city|              family|      date|store_nbr|sales|onpromotion|cluster|is_holiday|dcoilwtico|       hash_features|strIndxer_family|strIndxer_city|strIndxer_state|strIndxer_type|day_of_week|day_of_month|month|year|is_salary_day|
+----+------+------+--------------------+----------+---------+-----+-----------+-------+----------+----------+--------------------+----------------+--------------+---------------+--------------+-----------+------------+-----+----+-------------+
|   D| Azuay|Cuenca|SCHOOL AND OFFICE...|2013-01-01|       37|  0.0|          0|      2|         1|     93.14|(1024,[124,560,79...|              27|            14|             15|             4|          3|      

In [2]:
# 1. Start with RAW data
df_raw = df

# 2. CREATE LAG FEATURES FIRST (on full data!)
from pyspark.sql.window import Window

window_store_family = Window.partitionBy("store_nbr", "family").orderBy("date")
window_7d = Window.partitionBy("store_nbr", "family").orderBy("date").rowsBetween(-7, -1)
window_14d = Window.partitionBy("store_nbr", "family").orderBy("date").rowsBetween(-14, -1)

print("Creating lag features...")
df_with_lags = df_raw \
    .withColumn("sales_lag_7", F.lag("sales", 7).over(window_store_family)) \
    .withColumn("sales_lag_14", F.lag("sales", 14).over(window_store_family)) \
    .withColumn("sales_lag_30", F.lag("sales", 30).over(window_store_family)) \
    .withColumn("sales_rolling_mean_7", F.avg("sales").over(window_7d)) \
    .withColumn("sales_rolling_mean_14", F.avg("sales").over(window_14d)) \
    .withColumn("sales_rolling_std_7", F.stddev("sales").over(window_7d)) \
    .withColumn("promo_rolling_sum_7", F.sum("onpromotion").over(window_7d))

# Add date features
df_with_lags = df_with_lags \
    .withColumn("week_of_year", F.weekofyear("date")) \
    .withColumn("quarter", F.quarter("date")) \
    .withColumn("is_weekend", F.when(F.col("day_of_week").isin([5, 6]), 1).otherwise(0)) \
    .withColumn("is_month_start", F.when(F.dayofmonth("date") <= 5, 1).otherwise(0)) \
    .withColumn("is_month_end", F.when(F.dayofmonth("date") >= 25, 1).otherwise(0))

# 3. NOW find stable periods
window_14d_density = Window.partitionBy("store_nbr", "family").orderBy("date").rowsBetween(-13, 0)

df_with_density = df_with_lags.withColumn(
    "sales_days_in_window",
    F.sum(F.when(F.col("sales") > 0, 1).otherwise(0)).over(window_14d_density)
)

stable_starts = df_with_density.filter(F.col("sales_days_in_window") >= 7) \
    .groupBy("store_nbr", "family") \
    .agg(F.min("date").alias("stable_start_date"))

# 4. Filter to stable periods
df_with_stable = df_with_lags.join(
    stable_starts.select("store_nbr", "family", "stable_start_date"), 
    on=["store_nbr", "family"], 
    how="inner"
)

df_stable = df_with_stable.filter(F.col("date") >= F.col("stable_start_date"))

# 5. Require sufficient data
sufficient_data = df_stable.groupBy("store_nbr", "family").agg(
    F.countDistinct("date").alias("stable_days"),
    F.sum("sales").alias("total_sales")
).filter(
    (F.col("stable_days") >= 365) &
    (F.col("total_sales") >= 0)
)

df_trainable = df_stable.join(
    sufficient_data.select("store_nbr", "family"),
    on=["store_nbr", "family"],
    how="inner"
).drop("stable_start_date", "sales_days_in_window")

print(f"Trainable rows: {df_trainable.count():,}")

# 6. Verify lag features exist
print("\nVerifying lag features:")
for col in ['sales_lag_7', 'sales_lag_14', 'sales_rolling_mean_7']:
    if col in df_trainable.columns:
        null_count = df_trainable.filter(F.col(col).isNull()).count()
        print(f"✓ {col}: {null_count:,} nulls")
    else:
        print(f"❌ {col}: MISSING!")

Creating lag features...


Trainable rows: 2,369,333

Verifying lag features:


✓ sales_lag_7: 15 nulls


✓ sales_lag_14: 5,849 nulls


✓ sales_rolling_mean_7: 0 nulls


In [3]:
df = df_trainable.filter(F.col('family').isin(['HOME AND KITCHEN I', 'PRODUCE']))
df.show(2)
df.columns

25/12/24 01:56:44 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+------------------+----+---------+-----+----------+-----+-----------+-------+----------+----------+--------------------+----------------+--------------+---------------+--------------+-----------+------------+-----+----+-------------+-----------+------------+------------+--------------------+---------------------+-------------------+-------------------+------------+-------+----------+--------------+------------+
|store_nbr|            family|type|    state| city|      date|sales|onpromotion|cluster|is_holiday|dcoilwtico|       hash_features|strIndxer_family|strIndxer_city|strIndxer_state|strIndxer_type|day_of_week|day_of_month|month|year|is_salary_day|sales_lag_7|sales_lag_14|sales_lag_30|sales_rolling_mean_7|sales_rolling_mean_14|sales_rolling_std_7|promo_rolling_sum_7|week_of_year|quarter|is_weekend|is_month_start|is_month_end|
+---------+------------------+----+---------+-----+----------+-----+-----------+-------+----------+----------+--------------------+----------------+

['store_nbr',
 'family',
 'type',
 'state',
 'city',
 'date',
 'sales',
 'onpromotion',
 'cluster',
 'is_holiday',
 'dcoilwtico',
 'hash_features',
 'strIndxer_family',
 'strIndxer_city',
 'strIndxer_state',
 'strIndxer_type',
 'day_of_week',
 'day_of_month',
 'month',
 'year',
 'is_salary_day',
 'sales_lag_7',
 'sales_lag_14',
 'sales_lag_30',
 'sales_rolling_mean_7',
 'sales_rolling_mean_14',
 'sales_rolling_std_7',
 'promo_rolling_sum_7',
 'week_of_year',
 'quarter',
 'is_weekend',
 'is_month_start',
 'is_month_end']

In [4]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# IMPORTANT: Work with a manageable sample first
df_sample = df_trainable.sample(fraction=1.0, seed=42)  # Use 10% of data
print(f"Sample size: {df_sample.count()}")

# Define feature columns (only the most important ones to reduce memory)
feature_cols = ['strIndxer_family',
    'onpromotion', 'is_holiday', 'day_of_week', 'month', 'year',
    'sales_lag_7', 'sales_lag_14', 'sales_rolling_mean_7',
    'promo_rolling_sum_7', 'week_of_year', 'is_weekend'
]

# Keep only existing columns
feature_cols = [col for col in feature_cols if col in df_sample.columns]

# Remove nulls
df_clean = df_sample.select(['date', 'sales'] + feature_cols).na.drop()

# Repartition to reduce memory pressure
df_clean = df_clean.repartition(4)

print(f"Clean sample size: {df_clean.count()}")

# Assemble features
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

df_assembled = assembler.transform(df_clean)
df_assembled.show(5)

# Get date range
dates = df_assembled.agg(F.min("date"), F.max("date")).collect()[0]
print(f"Date range: {dates[0]} to {dates[1]}")

# Simple date split
train_df = df_assembled.limit(int(df_assembled.count() * 0.8))
test_df = df_assembled.subtract(train_df)

print(f"Train: {train_df.count()}, Test: {test_df.count()}")

# Use Random Forest (lighter than GBT)
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="sales",
    numTrees=20,  # Reduced from default
    maxDepth=5,   # Reduced depth
    seed=42
)

print("\nTraining model...")
model = rf.fit(train_df)

# Predict
predictions = model.transform(test_df)

# Evaluate
evaluator = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

evaluator_mae = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)

print(f"\n=== Results on 10% Sample ===")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

# Show sample predictions
predictions.select("date", "sales", "prediction").show(10)

Sample size: 2369333


Clean sample size: 2363484


+----------+------+----------------+-----------+----------+-----------+-----+----+-----------+------------+--------------------+-------------------+------------+----------+--------------------+
|      date| sales|strIndxer_family|onpromotion|is_holiday|day_of_week|month|year|sales_lag_7|sales_lag_14|sales_rolling_mean_7|promo_rolling_sum_7|week_of_year|is_weekend|            features|
+----------+------+----------------+-----------+----------+-----------+-----+----+-----------+------------+--------------------+-------------------+------------+----------+--------------------+
|2015-12-13| 490.0|              26|          0|         0|          1|   12|2015|      514.0|       419.0|   316.2857142857143|                  4|          50|         0|[26.0,0.0,0.0,1.0...|
|2015-05-28|   2.0|              11|          0|         0|          5|    5|2015|        1.0|         1.0|  0.7142857142857143|                  0|          22|         1|[11.0,0.0,0.0,5.0...|
|2015-02-27|1067.0|           

Date range: 2013-01-15 to 2017-08-15


Train: 1890787, Test: 436850

Training model...



=== Results on 10% Sample ===
RMSE: 482.02
MAE: 121.59


+----------+---------+------------------+
|      date|    sales|        prediction|
+----------+---------+------------------+
|2013-01-15|   66.642|40.401009108090335|
|2013-01-15|     95.0| 75.37626932353274|
|2013-01-15|    141.0| 97.63842771795456|
|2013-01-15|    156.0|100.06129060578576|
|2013-01-16|  489.326|511.82928773379325|
|2013-01-16|502.45798|1631.8953649837717|
|2013-01-18|      0.0| 26.02617578820253|
|2013-01-19|   1310.0|1088.3451383793213|
|2013-01-21|      6.0| 26.02617578820253|
|2013-01-21|  184.206|  135.054084640947|
+----------+---------+------------------+
only showing top 10 rows


In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import avg, sqrt, sum as spark_sum, count, abs as spark_abs, col

def evaluate_regression_model(predictions_df, label_col='sales', prediction_col='prediction', verbose=True):
    """
    Evaluate regression model predictions with comprehensive metrics.
    
    Parameters:
    -----------
    predictions_df : pyspark.sql.DataFrame
        DataFrame containing actual and predicted values
    label_col : str
        Name of the column with actual values (default: 'sales')
    prediction_col : str
        Name of the column with predicted values (default: 'prediction')
    verbose : bool
        Whether to print metrics (default: True)
    
    Returns:
    --------
    dict : Dictionary containing all evaluation metrics
    """
    
    # Overall metrics using RegressionEvaluator
    evaluator_rmse = RegressionEvaluator(
        labelCol=label_col,
        predictionCol=prediction_col,
        metricName='rmse'
    )
    
    evaluator_mae = RegressionEvaluator(
        labelCol=label_col,
        predictionCol=prediction_col,
        metricName='mae'
    )
    
    evaluator_r2 = RegressionEvaluator(
        labelCol=label_col,
        predictionCol=prediction_col,
        metricName='r2'
    )
    
    evaluator_mse = RegressionEvaluator(
        labelCol=label_col,
        predictionCol=prediction_col,
        metricName='mse'
    )
    
    # Calculate metrics
    rmse = evaluator_rmse.evaluate(predictions_df)
    mae = evaluator_mae.evaluate(predictions_df)
    r2 = evaluator_r2.evaluate(predictions_df)
    mse = evaluator_mse.evaluate(predictions_df)
    
    # Additional custom metrics
    stats = predictions_df.agg(
        count(label_col).alias('count'),
        avg(label_col).alias('mean_actual'),
        avg(prediction_col).alias('mean_predicted'),
        sqrt(avg((col(label_col) - col(prediction_col)) ** 2)).alias('rmse_check'),
        avg(spark_abs(col(label_col) - col(prediction_col))).alias('mae_check'),
        avg(spark_abs((col(label_col) - col(prediction_col)) / col(label_col)) * 100).alias('mape')
    ).collect()[0]
    
    # Compile results
    metrics = {
        'rmse': rmse,
        'mae': mae,
        'mse': mse,
        'r2': r2,
        'mape': stats['mape'],
        'count': stats['count'],
        'mean_actual': stats['mean_actual'],
        'mean_predicted': stats['mean_predicted']
    }
    
    if verbose:
        print("\n" + "="*60)
        print("REGRESSION MODEL EVALUATION METRICS")
        print("="*60)
        print(f"Sample Count:        {metrics['count']:,}")
        print(f"Mean Actual:         {metrics['mean_actual']:.2f}")
        print(f"Mean Predicted:      {metrics['mean_predicted']:.2f}")
        print("-"*60)
        print(f"RMSE:                {metrics['rmse']:.4f}")
        print(f"MAE:                 {metrics['mae']:.4f}")
        print(f"MSE:                 {metrics['mse']:.4f}")
        print(f"R² Score:            {metrics['r2']:.4f}")
        print(f"MAPE:                {metrics['mape']:.2f}%")
        print("="*60 + "\n")
    
    return metrics


def evaluate_by_group(predictions_df, group_cols, label_col='sales', prediction_col='prediction', top_n=10):
    """
    Calculate metrics for each group (e.g., by store_nbr and family).
    
    Parameters:
    -----------
    predictions_df : pyspark.sql.DataFrame
        DataFrame containing actual and predicted values
    group_cols : list or str
        Column(s) to group by (e.g., ['store_nbr', 'family'])
    label_col : str
        Name of the column with actual values (default: 'sales')
    prediction_col : str
        Name of the column with predicted values (default: 'prediction')
    top_n : int
        Number of top/bottom groups to display (default: 10)
    
    Returns:
    --------
    pyspark.sql.DataFrame : Metrics by group
    """
    
    if isinstance(group_cols, str):
        group_cols = [group_cols]
    
    metrics_by_group = predictions_df.groupBy(*group_cols).agg(
        count(label_col).alias('sample_count'),
        avg(label_col).alias('avg_actual'),
        avg(prediction_col).alias('avg_predicted'),
        sqrt(avg((col(label_col) - col(prediction_col)) ** 2)).alias('rmse'),
        avg(spark_abs(col(label_col) - col(prediction_col))).alias('mae'),
        avg(spark_abs((col(label_col) - col(prediction_col)) / col(label_col)) * 100).alias('mape')
    ).orderBy('rmse', ascending=False)
    
    print(f"\n{'='*80}")
    print(f"METRICS BY GROUP: {', '.join(group_cols)}")
    print(f"{'='*80}")
    print(f"\nTop {top_n} groups with HIGHEST error (RMSE):")
    print("-"*80)
    metrics_by_group.show(top_n, truncate=False)
    
    print(f"\nTop {top_n} groups with LOWEST error (RMSE):")
    print("-"*80)
    metrics_by_group.orderBy('rmse', ascending=True).show(top_n, truncate=False)
    
    return metrics_by_group


def compare_models(predictions_dfs, model_names, label_col='sales', prediction_col='prediction'):
    """
    Compare metrics across multiple models.
    
    Parameters:
    -----------
    predictions_dfs : list
        List of prediction DataFrames from different models
    model_names : list
        List of model names corresponding to each DataFrame
    label_col : str
        Name of the column with actual values (default: 'sales')
    prediction_col : str
        Name of the column with predicted values (default: 'prediction')
    
    Returns:
    --------
    pyspark.sql.DataFrame : Comparison table of all models
    """
    from pyspark.sql import SparkSession
    
    spark = SparkSession.builder.getOrCreate()
    
    results = []
    
    for name, pred_df in zip(model_names, predictions_dfs):
        metrics = evaluate_regression_model(pred_df, label_col, prediction_col, verbose=False)
        metrics['model_name'] = name
        results.append(metrics)
    
    # Convert to Spark DataFrame
    comparison_df = spark.createDataFrame(results)
    
    print("\n" + "="*100)
    print("MODEL COMPARISON")
    print("="*100)
    comparison_df.select(
        'model_name', 'rmse', 'mae', 'r2', 'mape', 'count'
    ).show(truncate=False)
    
    return comparison_df


def plot_predictions_sample(predictions_df, label_col='sales', prediction_col='prediction', sample_size=1000):
    """
    Display a sample of actual vs predicted values for quick inspection.
    
    Parameters:
    -----------
    predictions_df : pyspark.sql.DataFrame
        DataFrame containing actual and predicted values
    label_col : str
        Name of the column with actual values (default: 'sales')
    prediction_col : str
        Name of the column with predicted values (default: 'prediction')
    sample_size : int
        Number of samples to display (default: 1000)
    
    Returns:
    --------
    pyspark.sql.DataFrame : Sample of predictions with error metrics
    """
    
    sample_df = predictions_df.select(
        '*',
        (col(prediction_col) - col(label_col)).alias('error'),
        spark_abs(col(prediction_col) - col(label_col)).alias('abs_error'),
        (spark_abs((col(label_col) - col(prediction_col)) / col(label_col)) * 100).alias('pct_error')
    ).limit(sample_size)
    
    print(f"\n{'='*80}")
    print(f"SAMPLE PREDICTIONS (First {sample_size} rows)")
    print(f"{'='*80}")
    sample_df.select(
        label_col, prediction_col, 'error', 'abs_error', 'pct_error'
    ).show(20)
    
    return sample_df

In [9]:
# Example 1: Evaluate overall model performance
metrics = evaluate_regression_model(predictions)

# Example 2: Evaluate by store and family
group_metrics = evaluate_by_group(
    predictions, 
    group_cols=['store_nbr', 'family'],
    top_n=15
)

# Example 3: Evaluate by family only
family_metrics = evaluate_by_group(
    predictions,
    group_cols='family'
)

# Example 4: Compare multiple models
gbt_predictions = results_gbt['predictions']
rf_predictions = results_rf['predictions']
lr_predictions = results_lr['predictions']

comparison = compare_models(
    predictions_dfs=[gbt_predictions, rf_predictions, lr_predictions],
    model_names=['GBT', 'Random Forest', 'Linear Regression']
)

# Example 5: Quick inspection of predictions
sample = plot_predictions_sample(predictions, sample_size=500)

# Example 6: Silent evaluation (no printing)
metrics = evaluate_regression_model(predictions, verbose=False)
print(f"Model RMSE: {metrics['rmse']:.2f}")

25/12/24 02:00:36 ERROR Executor: Exception in task 0.0 in stage 459.0 (TID 1056)
org.apache.spark.SparkArithmeticException: [DIVIDE_BY_ZERO] Division by zero. Use `try_divide` to tolerate divisor being 0 and return NULL instead. If necessary set "spark.sql.ansi.enabled" to "false" to bypass this error. SQLSTATE: 22012
== DataFrame ==
"__truediv__" was called from
line 62 in cell [9]

	at org.apache.spark.sql.errors.QueryExecutionErrors$.divideByZeroError(QueryExecutionErrors.scala:205)
	at org.apache.spark.sql.errors.QueryExecutionErrors.divideByZeroError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage31.hashAgg_subExpr_4$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage31.hashAgg_doConsume_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage31.hashAgg_doAggregateWithKeysOutput_0$(Unknown Sour

ArithmeticException: [DIVIDE_BY_ZERO] Division by zero. Use `try_divide` to tolerate divisor being 0 and return NULL instead. If necessary set "spark.sql.ansi.enabled" to "false" to bypass this error. SQLSTATE: 22012
== DataFrame ==
"__truediv__" was called from
line 62 in cell [9]


In [ ]:
from pyspark.sql import functions as F

print("="*60)
print("GRANULARITY ANALYSIS")
print("="*60)

# Total rows
total_rows = df.count()
print(f"\nTotal rows: {total_rows:,}")

# Test different combinations
print("\nTesting unique combinations:")

# 1. Date level
date_count = df.select("date").distinct().count()
print(f"  Unique dates: {date_count:,}")

# 2. Date + Store
date_store = df.select("date", "store_nbr").distinct().count()
print(f"  Unique (date, store_nbr): {date_store:,}")

# 3. Date + Family
date_family = df.select("date", "family").distinct().count()
print(f"  Unique (date, family): {date_family:,}")

# 4. Date + Store + Family (likely the grain)
date_store_family = df.select("date", "store_nbr", "family").distinct().count()
print(f"  Unique (date, store_nbr, family): {date_store_family:,}")

print("\n" + "="*60)

# Determine granularity
if date_store_family == total_rows:
    print("✓ GRANULARITY: One row per (date, store_nbr, family)")
    print("  This means: Daily sales for each product family at each store")
else:
    print(f"⚠️  DUPLICATES DETECTED!")
    print(f"  Expected unique combinations: {date_store_family:,}")
    print(f"  Actual rows: {total_rows:,}")
    print(f"  Duplicate rows: {total_rows - date_store_family:,}")
    
    # Show example duplicates
    print("\nSample duplicates:")
    df.groupBy("date", "store_nbr", "family").count() \
      .filter(F.col("count") > 1) \
      .orderBy(F.desc("count")) \
      .show(5)

# Show store attributes relationship
print("\n" + "="*60)
print("STORE ATTRIBUTES (should be many-to-one with store_nbr):")
print("="*60)

store_attrs = df.select("store_nbr", "type", "state", "city").distinct()
store_count = df.select("store_nbr").distinct().count()
store_attrs_count = store_attrs.count()

print(f"Unique stores: {store_count}")
print(f"Unique (store, type, state, city) combos: {store_attrs_count}")

if store_count == store_attrs_count:
    print("✓ Each store has one unique (type, state, city)")
else:
    print("⚠️  Some stores have multiple type/state/city values")

# Sample data structure
print("\n" + "="*60)
print("SAMPLE DATA:")
print("="*60)
df.select("date", "store_nbr", "family", "sales", "type", "city", "state", "cluster").show(10, truncate=False)

In [ ]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Feature columns (excluding family since we're grouping by it)


feature_cols = [
 'onpromotion',
 'cluster',
 'is_holiday',
 'dcoilwtico',
#  'hash_features',
 'strIndxer_family',
 'strIndxer_city',
 'strIndxer_state',
 'strIndxer_type',
 'day_of_week',
 'day_of_month',
 'month',
 'year',
 'is_salary_day',
 'sales_lag_7',
 'sales_lag_14',
 'sales_lag_30',
 'sales_rolling_mean_7',
 'sales_rolling_mean_14',
 'sales_rolling_std_7',
 'promo_rolling_sum_7',
 'week_of_year',
 'quarter',
 'is_weekend',
 'is_month_start',
 'is_month_end']

# feature_cols = [
#     'onpromotion', 'is_holiday', 'day_of_week', 'month', 'year',
#     'sales_lag_7', 'sales_lag_14', 'sales_rolling_mean_7',
#     'promo_rolling_sum_7', 'week_of_year', 'is_weekend',
#     'strIndxer_city', 'strIndxer_state', 'strIndxer_type', 'cluster'
#     # Note: NOT including strIndxer_family since we're training per family
# ]

feature_cols = [col for col in feature_cols if col in df.columns]

# Get unique families
families = df.select("family").distinct().rdd.flatMap(lambda x: x).collect()
print(f"Training models for {len(families)} families")

# Dictionary to store models and metrics
models = {}
metrics = []

# Get date range for splitting
dates = df.agg(F.min("date"), F.max("date")).collect()[0]
split_date = dates[0] + (dates[1] - dates[0]) * 0.8

for family in families:
    print(f"\n{'='*50}")
    print(f"Training model for: {family}")
    print(f"{'='*50}")
    
    # Filter data for this family
    df_family = df.filter(F.col("family") == family)
    
    # Check if sufficient data
    count = df_family.count()
    if count < 1000:  # Skip if too little data
        print(f"Skipping {family} - insufficient data ({count} rows)")
        continue
    
    print(f"Rows: {count}")
    
    # Clean and prepare features
    df_clean = df_family.select(['date', 'sales'] + feature_cols).na.drop()
    
    assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
    df_assembled = assembler.transform(df_clean)
    
    # Time-based split
    train_df = df_assembled.filter(F.col("date") < split_date).cache()
    test_df = df_assembled.filter(F.col("date") >= split_date).cache()
    
    train_count = train_df.count()
    test_count = test_df.count()
    print(f"Train: {train_count}, Test: {test_count}")
    
    if train_count < 100 or test_count < 10:
        print(f"Skipping {family} - insufficient train/test data")
        train_df.unpersist()
        test_df.unpersist()
        continue
    
    # Train model
    rf = RandomForestRegressor(
        featuresCol="features",
        labelCol="sales",
        numTrees=30,
        maxDepth=6,
        seed=42
    )
    
    try:
        model = rf.fit(train_df)
        
        # Predict
        predictions = model.transform(test_df)
        
        # Evaluate
        evaluator_rmse = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="rmse")
        evaluator_mae = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="mae")
        evaluator_r2 = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="r2")
        
        rmse = evaluator_rmse.evaluate(predictions)
        mae = evaluator_mae.evaluate(predictions)
        r2 = evaluator_r2.evaluate(predictions)
        
        print(f"RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.4f}")
        
        # Store results
        models[family] = model
        metrics.append({
            'family': family,
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'train_rows': train_count,
            'test_rows': test_count
        })
        
        # Save model
        # model.write().overwrite().save(f"/Volumes/portfolio_catalog/databricks_pipeline/models/rf_sales_{family.replace(' ', '_')}")
        
    except Exception as e:
        print(f"Error training model for {family}: {e}")
    
    finally:
        train_df.unpersist()
        test_df.unpersist()


In [ ]:

# Summary of all models
print(f"\n{'='*50}")
print("SUMMARY OF ALL MODELS")
print(f"{'='*50}")

import pandas as pd
metrics_df = pd.DataFrame(metrics).sort_values('r2')
print(metrics_df.to_string(index=False))

print(f"\nAverage RMSE: {metrics_df['rmse'].mean():.2f}")
print(f"Average MAE: {metrics_df['mae'].mean():.2f}")
print(f"Average R²: {metrics_df['r2'].mean():.4f}")
    
# Save metrics
metrics_df.to_csv('./model_metrics_families_all_periods.csv', index=False)
print(f"\n✓ Metrics saved to ./model_metrics_families_all_periods.csv")

In [ ]:
dfx = df.filter(F.col('family') == 'HOME AND KITCHEN I').groupBy('date', 'store_nbr', 'family').agg(F.sum('sales').alias('sales')).toPandas()
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12, 4))
sns.lineplot(data=dfx, x='date', y='sales', hue='store_nbr', alpha=0.5)
plt.title('HOME AND KITCHEN I');

In [ ]:
dfx = df.filter(F.col('family') == 'PRODUCE').groupBy('date', 'store_nbr', 'family').agg(F.sum('sales').alias('sales')).toPandas()
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12, 4))
sns.lineplot(data=dfx, x='date', y='sales', hue='store_nbr', alpha=0.5)
plt.title('PRODUCE');

In [ ]:
dfx_weekly = (
    dfx
    .assign(date=pd.to_datetime(dfx["date"]))
    .set_index("date")
    .groupby("family")["sales"]
    .resample("W")
    .sum()
    .reset_index()
)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.FacetGrid(
    dfx_weekly,
    col="family",
    col_wrap=5,
    height=1.4,
    aspect=3.0,     # wider figure
    sharex=True,
    sharey=False
)

g.map_dataframe(sns.lineplot, x="date", y="sales")

g.set_titles("{col_name}")
g.set_axis_labels("Week", "Weekly Sales")
g.fig.subplots_adjust(top=0.95)
g.fig.suptitle("Weekly Sales by Family", fontsize=16)

for ax in g.axes.flat:
    ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()


- We have some families that have late-starts, for example this one with sales from 2014. Also some period of no sales in 2014.
- Let's focus on the late-starts as the next step.

In [ ]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd

# Step 1: Identify first sale date for each store-family combination
store_family_start = df.filter(F.col("sales") > 0).groupBy("store_nbr", "family").agg(
    F.min("date").alias("active_start_date"),
    F.count("*").alias("active_days")
)

print(f"Store-family combinations with sales: {store_family_start.count()}")
store_family_start.show(50)

# Step 2: Join back to main df and filter
df_with_start = df.join(store_family_start, on=["store_nbr", "family"], how="inner")

# Only keep data from active_start_date onwards for each combo
df_active = df_with_start.filter(F.col("date") >= F.col("active_start_date"))

print(f"\nOriginal rows: {df.count():,}")
print(f"Active period rows: {df_active.count():,}")

# Step 3: Filter out combinations with insufficient data
# Require at least 365 days of data for training
sufficient_data = df_active.groupBy("store_nbr", "family").agg(
    F.countDistinct("date").alias("days_count")
).filter(F.col("days_count") >= 365)

print(f"Combinations with >=365 days: {sufficient_data.count()}")

df_trainable = df_active.join(
    sufficient_data.select("store_nbr", "family"),
    on=["store_nbr", "family"],
    how="inner"
).drop("active_start_date", "active_days")

print(f"Final trainable rows: {df_trainable.count():,}")

# Now train models per family as before, but using df_trainable
feature_cols = [
    'onpromotion', 'is_holiday', 'day_of_week', 'month', 'year',
    'sales_lag_7', 'sales_lag_14', 'sales_rolling_mean_7',
    'promo_rolling_sum_7', 'week_of_year', 'is_weekend',
    'strIndxer_city', 'strIndxer_state', 'strIndxer_type', 'cluster'
    # Note: NOT including strIndxer_family since we're training per family
]

feature_cols = [col for col in feature_cols if col in df_trainable.columns]

# Get families
families = [row['family'] for row in df_trainable.select("family").distinct().collect()]
print(f"\nTraining models for {len(families)} families")

all_metrics = []
successful_models = 0

for idx, family in enumerate(families, 1):
    print(f"\n{'='*60}")
    print(f"[{idx}/{len(families)}] Training: {family}")
    print(f"{'='*60}")
    
    try:
        # Filter for this family
        df_family = df_trainable.filter(F.col("family") == family)
        
        # For each store-family, we're already using only their active period
        count = df_family.count()
        print(f"Total rows: {count:,}")
        
        if count < 1000:
            print(f"⚠️  Skipping - insufficient data")
            continue
        
        # Clean data
        df_clean = df_family.select(['date', 'sales', 'store_nbr'] + feature_cols).na.drop()
        # df_clean = df_family.select(['date', 'sales'] + feature_cols).na.drop()
        clean_count = df_clean.count()
        print(f"Clean rows: {clean_count:,}")
        
        # Get date range for THIS family's data
        date_range = df_clean.agg(F.min("date"), F.max("date")).collect()[0]
        family_min_date = date_range[0]
        family_max_date = date_range[1]
        
        # Calculate 80/20 split based on THIS family's date range
        date_span = (family_max_date - family_min_date).days
        split_date = family_min_date + pd.Timedelta(days=int(date_span * 0.8))
        
        print(f"Date range: {family_min_date} to {family_max_date}")
        print(f"Split date: {split_date}")
        
        # Assemble features
        assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
        df_assembled = assembler.transform(df_clean)
        
        # Time-based split
        train_df = df_assembled.filter(F.col("date") < split_date).cache()
        test_df = df_assembled.filter(F.col("date") >= split_date).cache()
        
        train_count = train_df.count()
        test_count = test_df.count()
        print(f"Train: {train_count:,} | Test: {test_count:,}")
        
        if train_count < 100 or test_count < 20:
            print(f"⚠️  Skipping - insufficient split data")
            train_df.unpersist()
            test_df.unpersist()
            continue
        
        # Train model
        rf = RandomForestRegressor(
            featuresCol="features",
            labelCol="sales",
            numTrees=50,
            maxDepth=8,
            seed=42
        )
        
        print("Training model...")
        model = rf.fit(train_df)
        
        print("Making predictions...")
        predictions = model.transform(test_df)
        
        # Evaluate
        evaluator_rmse = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="rmse")
        evaluator_mae = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="mae")
        evaluator_r2 = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="r2")
        
        rmse = evaluator_rmse.evaluate(predictions)
        mae = evaluator_mae.evaluate(predictions)
        r2 = evaluator_r2.evaluate(predictions)
        
        print(f"✓ Results: RMSE={rmse:.2f} | MAE={mae:.2f} | R²={r2:.4f}")
        
        # Save metrics
        all_metrics.append({
            'family': family,
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'train_rows': train_count,
            'test_rows': test_count,
            'date_range_days': date_span
        })
        
        successful_models += 1
        
        # Clean up
        train_df.unpersist()
        test_df.unpersist()
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")



In [ ]:
# Final summary
print(f"\n{'='*60}")
print("TRAINING COMPLETE")
print(f"{'='*60}")
print(f"Successfully trained: {successful_models}/{len(families)} models")

if all_metrics:
    metrics_df = pd.DataFrame(all_metrics).sort_values('r2')
    print("\n" + metrics_df.to_string(index=False))
    
    print(f"\n📊 Overall Performance:")
    print(f"   Average RMSE: {metrics_df['rmse'].mean():.2f}")
    print(f"   Average MAE: {metrics_df['mae'].mean():.2f}")
    print(f"   Average R²: {metrics_df['r2'].mean():.4f}")
    
    # Save metrics
    metrics_df.to_csv('./model_metrics_active_periods.csv', index=False)
    print(f"\n✓ Metrics saved to ./model_metrics_active_periods.csv")

Our average R² increased from 0.4151 to 0.5013.

In [ ]:
import os
model_metrics = os.listdir('.')

metrics_dfs = []
for model in model_metrics:
    if model.startswith('model_metrics'):
        print(f"Model metrics: {model}")
        metrics_df = pd.read_csv(model)
        metrics_df['model'] = model.replace('.csv', '')
        metrics_dfs.append(metrics_df)
        # print(metrics_df.to_string(index=False))

metrics_dfs = pd.concat(metrics_dfs)

In [ ]:
data = (
    metrics_dfs
    .pivot_table(
        index='family',
        columns='model',
        values='r2',
        aggfunc='mean'
    )
    .reset_index()
    .round(2).sort_values('model_metrics_active_periods', ascending=False)
)

long = data.melt(
    id_vars='family',
    value_vars=[
        'model_metrics_families_all_periods',
        'model_metrics_active_periods'
    ],
    var_name='period',
    value_name='r2'
)

fig, ax = plt.subplots(figsize=(20, 4))

sns.barplot(
    data=long,
    x='family',
    y='r2',
    hue='period',
    palette=['lightblue', 'lightgreen']
)

plt.xticks(rotation=90)
plt.ylabel('R²')
plt.xlabel('')
plt.legend(title='');

- There are some improvements by removing the late-start periods.
- However, we also noticed there are some issues with large date gaps. Let's try to fix those next.
- And let's focus on HOME AND KITCHEN I

In [ ]:
dfx = df.filter(F.col('family') == 'HOME AND KITCHEN I').groupBy('date', 'store_nbr', 'family').agg(F.sum('sales').alias('sales')).toPandas()

dfx_weekly = (
    dfx
    .assign(date=pd.to_datetime(dfx["date"]))
    .groupby(["family", "store_nbr"])  # include store
    .resample("W", on="date")      # resample weekly
    .sales
    .sum()
    .reset_index()
)


In [ ]:

family_name = "HOME AND KITCHEN I"

g = sns.FacetGrid(
    dfx_weekly[dfx_weekly["family"] == family_name],
    col="store_nbr",
    col_wrap=5,
    height=1.4,
    aspect=3.0,
    sharex=True,
    sharey=False
)

g.map_dataframe(sns.lineplot, x="date", y="sales")

g.set_titles("Store {col_name}")
g.set_axis_labels("Week", "Weekly Sales")

for ax in g.axes.flat:
    ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()


There are some stores with late-start after 2015. However, our previous model started from 2014-01-01.


- [19/32] Training: HOME AND KITCHEN I
- Total rows: 67,076
- Clean rows: 67,076
- Date range: 2014-01-01 to 2017-08-15
- Split date: 2016-11-23
- Train: 53,031 | Test: 14,045


In [ ]:
# df = spark.read.parquet('../notebooks/data/train.parquet')
# df = df.filter(F.col('family') == 'HOME AND KITCHEN I')
# df.show(4)

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Phase 1: Find "stable start date" for each store-family
# Define stable as: at least 7 consecutive days with sales in a 10-day window

window_10d = Window.partitionBy("store_nbr", "family").orderBy("date").rowsBetween(-9, 0)

df_with_density = df.withColumn(
    "sales_in_last_10_days",
    F.sum(F.when(F.col("sales") > 0, 1).otherwise(0)).over(window_10d)
)

# Mark where stable period begins (7+ sales days in last 10 days)
df_stable_flag = df_with_density.withColumn(
    "is_stable",
    F.when(F.col("sales_in_last_10_days") >= 7, 1).otherwise(0)
)

# Get first stable date for each store-family
stable_starts = df_stable_flag.filter(F.col("is_stable") == 1) \
    .groupBy("store_nbr", "family") \
    .agg(F.min("date").alias("stable_start_date"))

print("Sample stable start dates:")
stable_starts.show(54)

# Phase 2: Use only stable period data
df_stable = df.join(stable_starts, on=["store_nbr", "family"], how="inner") \
              .filter(F.col("date") >= F.col("stable_start_date"))

print(f"\nOriginal rows: {df.count():,}")
print(f"Stable period rows: {df_stable.count():,}")

# Phase 3: Require minimum data for training
sufficient_data = df_stable.groupBy("store_nbr", "family").agg(
    F.countDistinct("date").alias("days_count"),
    F.sum("sales").alias("total_sales")
).filter(
    (F.col("days_count") >= 365) &  # At least 1 year of STABLE data
    (F.col("total_sales") >= 0)
)

print(f"Store-family combos with sufficient stable data: {sufficient_data.count()}")

df_trainable = df_stable.join(
    sufficient_data.select("store_nbr", "family"),
    on=["store_nbr", "family"],
    how="inner"
).drop("stable_start_date")

print(f"Final trainable rows: {df_trainable.count():,}")
df_trainable.show(4)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd

# feature_cols = [
#     'onpromotion', 'is_holiday', 'day_of_week', 'day_of_month', 'month', 'year',
#     'sales_lag_7', 'sales_lag_14', 'sales_lag_30',
#     'sales_rolling_mean_7', 'sales_rolling_mean_14', 'sales_rolling_std_7',
#     'promo_rolling_sum_7', 'week_of_year', 'quarter', 'is_weekend',
#     'is_month_start', 'is_month_end', 'is_salary_day',
#     'strIndxer_city', 'strIndxer_state', 'strIndxer_type', 'cluster'
# ]

feature_cols = [
 'onpromotion',
 'cluster',
 'is_holiday',
 'dcoilwtico',
#  'hash_features',
 'strIndxer_family',
 'strIndxer_city',
 'strIndxer_state',
 'strIndxer_type',
 'day_of_week',
 'day_of_month',
 'month',
 'year',
 'is_salary_day',
 'sales_lag_7',
 'sales_lag_14',
 'sales_lag_30',
 'sales_rolling_mean_7',
 'sales_rolling_mean_14',
 'sales_rolling_std_7',
 'promo_rolling_sum_7',
 'week_of_year',
 'quarter',
 'is_weekend',
 'is_month_start',
 'is_month_end']


# feature_cols = [
#     'onpromotion', 'is_holiday', 'day_of_week', 'month', 'year',
#     'sales_lag_7', 'sales_lag_14', 'sales_rolling_mean_7',
#     'promo_rolling_sum_7', 'week_of_year', 'is_weekend',
#     'strIndxer_city', 'strIndxer_state', 'strIndxer_type', 'cluster'
#     # Note: NOT including strIndxer_family since we're training per family
# ]


feature_cols = [col for col in feature_cols if col in df_trainable.columns]

families = [row['family'] for row in df_trainable.select("family").distinct().collect()]
print(f"\nTraining models for {len(families)} families")

all_metrics = []

for idx, family in enumerate(families, 1):
    print(f"\n{'='*60}")
    print(f"[{idx}/{len(families)}] Training: {family}")
    print(f"{'='*60}")
    
    try:
        df_family = df_trainable.filter(F.col("family") == family)
        count = df_family.count()
        print(f"Total rows (stable periods only): {count:,}")
        
        if count < 1000:
            print(f"⚠️  Skipping - insufficient data")
            continue
        
        # Each store-family is already filtered to its stable period
        # Check date range
        date_range = df_family.agg(F.min("date"), F.max("date")).collect()[0]
        print(f"Date range: {date_range[0]} to {date_range[1]}")
        
        # Clean and prepare
        df_clean = df_family.select(['date', 'sales', 'store_nbr'] + feature_cols).na.drop()
        
        # Calculate split date (80/20)
        date_span = (date_range[1] - date_range[0]).days
        split_date = date_range[0] + pd.Timedelta(days=int(date_span * 0.8))
        print(f"Split date: {split_date}")
        
        assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
        df_assembled = assembler.transform(df_clean)
        
        train_df = df_assembled.filter(F.col("date") < split_date).cache()
        test_df = df_assembled.filter(F.col("date") >= split_date).cache()
        
        train_count = train_df.count()
        test_count = test_df.count()
        print(f"Train: {train_count:,} | Test: {test_count:,}")
        
        if train_count < 100 or test_count < 20:
            print(f"⚠️  Skipping - insufficient split")
            train_df.unpersist()
            test_df.unpersist()
            continue
        
        # Train
        rf = RandomForestRegressor(
            featuresCol="features",
            labelCol="sales",
            numTrees=50,
            maxDepth=8,
            seed=42
        )
        
        model = rf.fit(train_df)
        predictions = model.transform(test_df)
        
        # Evaluate
        rmse = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="rmse").evaluate(predictions)
        mae = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="mae").evaluate(predictions)
        r2 = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="r2").evaluate(predictions)
        
        print(f"✓ RMSE={rmse:.2f} | MAE={mae:.2f} | R²={r2:.4f}")
        
        all_metrics.append({
            'family': family,
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'train_rows': train_count,
            'test_rows': test_count
        })
        
        train_df.unpersist()
        test_df.unpersist()
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")

# Summary
if all_metrics:
    metrics_df = pd.DataFrame(all_metrics).sort_values('rmse')
    print("\n" + metrics_df.to_string(index=False))
    print(f"\n📊 Average RMSE: {metrics_df['rmse'].mean():.2f}")
    print(f"📊 Average R²: {metrics_df['r2'].mean():.4f}")
    metrics_df.to_csv('./metrics_stable_periods.csv', index=False)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd

feature_cols = [
 'onpromotion',
 'cluster',
 'is_holiday',
 'dcoilwtico',
#  'hash_features',
 'strIndxer_family',
 'strIndxer_city',
 'strIndxer_state',
 'strIndxer_type',
 'day_of_week',
 'day_of_month',
 'month',
 'year',
 'is_salary_day',
 'sales_lag_7',
 'sales_lag_14',
 'sales_lag_30',
 'sales_rolling_mean_7',
 'sales_rolling_mean_14',
 'sales_rolling_std_7',
 'promo_rolling_sum_7',
 'week_of_year',
 'quarter',
 'is_weekend',
 'is_month_start',
 'is_month_end']


# feature_cols = [
#     'onpromotion', 'is_holiday', 'day_of_week', 'month', 'year',
#     'sales_lag_7', 'sales_lag_14', 'sales_rolling_mean_7',
#     'promo_rolling_sum_7', 'week_of_year', 'is_weekend',
#     'strIndxer_city', 'strIndxer_state', 'strIndxer_type', 'cluster'
# ]

feature_cols = [col for col in feature_cols if col in df_trainable.columns]
print(f"Available features: {feature_cols}")

families = [row['family'] for row in df_trainable.select("family").distinct().collect()]
print(f"\nTraining models for {len(families)} families")

all_metrics = []
skipped_families = []

for idx, family in enumerate(families, 1):
    print(f"\n{'='*60}")
    print(f"[{idx}/{len(families)}] Training: {family}")
    print(f"{'='*60}")
    
    try:
        df_family = df_trainable.filter(F.col("family") == family)
        count = df_family.count()
        print(f"Total rows (stable periods only): {count:,}")
        
        if count < 1000:
            print(f"⚠️  Skipping - insufficient data")
            skipped_families.append({'family': family, 'reason': 'insufficient_data', 'rows': count})
            continue
        
        # Show sales statistics
        sales_stats = df_family.agg(
            F.min("sales").alias("min_sales"),
            F.avg("sales").alias("avg_sales"),
            F.max("sales").alias("max_sales"),
            F.stddev("sales").alias("std_sales")
        ).collect()[0]
        
        print(f"Sales stats: min={sales_stats['min_sales']:.1f}, avg={sales_stats['avg_sales']:.1f}, "
              f"max={sales_stats['max_sales']:.1f}, std={sales_stats['std_sales']:.1f}")
        
        # Check date range
        date_range = df_family.agg(F.min("date"), F.max("date")).collect()[0]
        print(f"Date range: {date_range[0]} to {date_range[1]}")
        
        # Clean and prepare
        df_clean = df_family.select(['date', 'sales', 'store_nbr'] + feature_cols).na.drop()
        clean_count = df_clean.count()
        print(f"After dropping nulls: {clean_count:,} rows ({100*clean_count/count:.1f}%)")
        
        if clean_count < 1000:
            print(f"⚠️  Skipping - too many nulls")
            skipped_families.append({'family': family, 'reason': 'too_many_nulls', 'rows': clean_count})
            continue
        
        # Calculate split date (80/20)
        date_span = (date_range[1] - date_range[0]).days
        split_date = date_range[0] + pd.Timedelta(days=int(date_span * 0.8))
        print(f"Split date: {split_date} ({date_span} days total)")
        
        assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
        df_assembled = assembler.transform(df_clean)
        
        train_df = df_assembled.filter(F.col("date") < split_date).cache()
        test_df = df_assembled.filter(F.col("date") >= split_date).cache()
        
        train_count = train_df.count()
        test_count = test_df.count()
        print(f"Train: {train_count:,} | Test: {test_count:,}")
        
        if train_count < 100 or test_count < 20:
            print(f"⚠️  Skipping - insufficient split")
            skipped_families.append({'family': family, 'reason': 'insufficient_split', 
                                    'train': train_count, 'test': test_count})
            train_df.unpersist()
            test_df.unpersist()
            continue
        
        # Train
        rf = RandomForestRegressor(
            featuresCol="features",
            labelCol="sales",
            numTrees=50,
            maxDepth=8,
            seed=42
        )
        
        print("Training model...")
        model = rf.fit(train_df)
        
        print("Making predictions...")
        predictions = model.transform(test_df)
        
        # Show sample predictions
        sample_preds = predictions.select("date", "sales", "prediction").orderBy("date").limit(10)
        print("\nSample predictions:")
        sample_preds.show(10, truncate=False)
        
        # Evaluate
        rmse = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="rmse").evaluate(predictions)
        mae = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="mae").evaluate(predictions)
        r2 = RegressionEvaluator(labelCol="sales", predictionCol="prediction", metricName="r2").evaluate(predictions)
        
        # Calculate additional metrics
        pred_stats = predictions.agg(
            F.avg("prediction").alias("avg_pred"),
            F.stddev("prediction").alias("std_pred")
        ).collect()[0]
        
        print(f"\n✓ Results:")
        print(f"  RMSE: {rmse:.2f}")
        print(f"  MAE: {mae:.2f}")
        print(f"  R²: {r2:.4f}")
        print(f"  Avg actual sales: {sales_stats['avg_sales']:.1f}")
        print(f"  Avg predicted sales: {pred_stats['avg_pred']:.1f}")
        
        # Feature importance
        feature_importances = model.featureImportances.toArray()
        importance_df = pd.DataFrame({
            'feature': feature_cols,
            'importance': feature_importances
        }).sort_values('importance', ascending=False)
        
        print(f"\nTop 5 features:")
        for _, row in importance_df.head(5).iterrows():
            print(f"  {row['feature']}: {row['importance']:.4f}")
        
        all_metrics.append({
            'family': family,
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'train_rows': train_count,
            'test_rows': test_count,
            'avg_sales': sales_stats['avg_sales'],
            'avg_prediction': pred_stats['avg_pred']
        })
        
        train_df.unpersist()
        test_df.unpersist()
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        skipped_families.append({'family': family, 'reason': 'error', 'error': str(e)})

# Summary
print(f"\n{'='*60}")
print("FINAL SUMMARY")
print(f"{'='*60}")

if all_metrics:
    metrics_df = pd.DataFrame(all_metrics).sort_values('r2', ascending=False)
    print("\n" + metrics_df.to_string(index=False))
    
    print(f"\n📊 Performance Statistics:")
    print(f"   Average RMSE: {metrics_df['rmse'].mean():.2f}")
    print(f"   Average MAE: {metrics_df['mae'].mean():.2f}")
    print(f"   Average R²: {metrics_df['r2'].mean():.4f}")
    print(f"   Median R²: {metrics_df['r2'].median():.4f}")
    print(f"   Best R²: {metrics_df['r2'].max():.4f} ({metrics_df.iloc[0]['family']})")
    print(f"   Worst R²: {metrics_df['r2'].min():.4f} ({metrics_df.iloc[-1]['family']})")
    
    # Save
    metrics_df.to_csv('./metrics_stable_periods.csv', index=False)
    print(f"\n✓ Metrics saved to ./metrics_stable_periods.csv")
else:
    print("❌ No models successfully trained!")

if skipped_families:
    print(f"\n⚠️  Skipped {len(skipped_families)} families:")
    skipped_df = pd.DataFrame(skipped_families)
    print(skipped_df.to_string(index=False))

In [ ]:
# Diagnostic 1: Check if test data overlaps with training somehow
print("=== CHECKING TRAIN/TEST SPLIT ===")

df_family = df_trainable.filter(F.col("family") == "HOME AND KITCHEN I")

date_range = df_family.agg(F.min("date"), F.max("date")).collect()[0]
print(f"Full date range: {date_range[0]} to {date_range[1]}")

split_date = date_range[0] + pd.Timedelta(days=int((date_range[1] - date_range[0]).days * 0.8))
print(f"Split date: {split_date}")

train_dates = df_family.filter(F.col("date") < split_date).agg(F.min("date"), F.max("date")).collect()[0]
test_dates = df_family.filter(F.col("date") >= split_date).agg(F.min("date"), F.max("date")).collect()[0]

print(f"Train: {train_dates[0]} to {train_dates[1]}")
print(f"Test: {test_dates[0]} to {test_dates[1]}")

# Diagnostic 2: Check sales distribution in train vs test
print("\n=== SALES DISTRIBUTION ===")
train_stats = df_family.filter(F.col("date") < split_date).agg(
    F.avg("sales").alias("avg"),
    F.stddev("sales").alias("std"),
    F.min("sales").alias("min"),
    F.max("sales").alias("max")
).collect()[0]

test_stats = df_family.filter(F.col("date") >= split_date).agg(
    F.avg("sales").alias("avg"),
    F.stddev("sales").alias("std"),
    F.min("sales").alias("min"),
    F.max("sales").alias("max")
).collect()[0]

print(f"Train sales: avg={train_stats['avg']:.1f}, std={train_stats['std']:.1f}, min={train_stats['min']:.1f}, max={train_stats['max']:.1f}")
print(f"Test sales:  avg={test_stats['avg']:.1f}, std={test_stats['std']:.1f}, min={test_stats['min']:.1f}, max={test_stats['max']:.1f}")

# Diagnostic 3: Check for nulls in lag features
print("\n=== NULL CHECK ===")
df_clean = df_family.select(['date', 'sales'] + feature_cols).na.drop()
print(f"Before na.drop(): {df_family.count():,} rows")
print(f"After na.drop(): {df_clean.count():,} rows")
print(f"Dropped: {df_family.count() - df_clean.count():,} rows ({100*(df_family.count() - df_clean.count())/df_family.count():.1f}%)")

# Diagnostic 4: Look at actual predictions
print("\n=== SAMPLE PREDICTIONS ===")
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_assembled = assembler.transform(df_clean)

train_df = df_assembled.filter(F.col("date") < split_date)
test_df = df_assembled.filter(F.col("date") >= split_date)

rf = RandomForestRegressor(featuresCol="features", labelCol="sales", numTrees=50, maxDepth=8, seed=42)
model = rf.fit(train_df)
predictions = model.transform(test_df)

predictions.select("date", "sales", "prediction", "sales_lag_7", "sales_rolling_mean_7") \
    .orderBy("date").show(30, truncate=False)

# Check for any extreme predictions
print("\n=== EXTREME PREDICTIONS ===")
predictions.select(
    F.min("prediction").alias("min_pred"),
    F.max("prediction").alias("max_pred"),
    F.avg("prediction").alias("avg_pred")
).show()